### Create Connectivity matrix

In [ ]:
def mesh_reader(file, coordinates,marker_elements, marker):
    # This method is for reading mesh files
    # indicated for 3D su2 files
    nelements = 0
    markers_id = []
    
    with open(coordinates, 'w') as fout, open(marker_elements, 'w') as fout2, open(file, 'r') as fin:
        while True:
            line = fin.readline()
            if not line:
                break
            
            if 'NPOIN=' in line:
                _, n_poin_str = line.split('=')
                n_poin = int(n_poin_str)
                print(f'total number of points {n_poin}')
                counter = 0
                while counter < n_poin:
                    line = fin.readline()
                    line_ = line.split()
                    fout.write(f'{line_[0]},{line_[1]},{line_[2]},{line_[3]}\n')
                    counter += 1

            
            if f'MARKER_TAG= {marker}' in line:
                print(line)
                line = fin.readline()
                line = line.split()
                nelements = int(line[1])
                print(f'number of marker tag elements {nelements}')
                for _ in range(nelements):
                    line = fin.readline()
                    line_ = line.split()
                    fout2.write(f'{line_[1]},{line_[2]},{line_[3]}\n')
                    # markers_id.append(int(line[1]))
                break
    
    fout.close()          
    fout2.close()

    coor = []
    mark_elem = []
    print('importing coordinates of whole model')
    
    with open(coordinates, 'r') as coordinates:
        for line in coordinates:
            row = line.split(',')
            coor.append([float(row[0]), float(row[1]),float(row[2]), int(row[3])])
    
    with open(marker_elements, 'r') as marker_elements:
        for line in marker_elements:
            row = line.split(',')
            mark_elem.append([int(row[0]), int(row[1]), int(row[2])])
    

    print('Mesh reader: process terminated')
    return coor, mark_elem


In [ ]:
mesh_file = 'Agard_446p5_v3.su2'
coordinates = 'coordinates.csv'
marker_elements = 'marker_elements.csv'

MARKERS = {'wing'}
for item in MARKERS:
    coor_, mark_elem_ = mesh_reader(mesh_file, coordinates,marker_elements, item)


In [ ]:
import pandas as pd

coordinates = pd.read_csv('coordinates.csv', header=None)
surface = pd.read_csv('surface.csv')
coord_surface = surface['PointID']

# Extract the last column from coordinates and filter it
last_column = coordinates.iloc[:, -1]  # Assuming the last column is the one you want
filtered_coordinates = coordinates[last_column.isin(coord_surface)]
point_id_reoder = list(range(len(filtered_coordinates.iloc[:, -1])))
filtered_coordinates.iloc[:, -1] = point_id_reoder
filtered_coordinates.to_csv('coordinates.csv', header=False, index=False)

In [ ]:
# Reorder the marker elements
import csv

# Read the CSV file
filename = 'marker_elements.csv'
data = []
with open(filename, 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        data.append(row)

# Create a mapping between original numbers and new numbers
unique_numbers = set()
for row in data:
    unique_numbers.update(map(int, row))
number_mapping = {number: idx for idx, number in enumerate(sorted(unique_numbers))}

# Read the CSV file again and replace numbers with new values
data = []
with open(filename, 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        new_row = [number_mapping[int(number)] for number in row]  # Convert each value to int
        data.append(new_row)

# Save the modified data to a new CSV file
modified_filename = 'marker_elements.csv'
with open(modified_filename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data)

### Write Connectivity matrix

In [ ]:
import numpy as np

# Read the file containing point connections with three columns
with open('marker_elements.csv', 'r') as conn_file:
    connections = [line.strip().split(',') for line in conn_file]


# Create a list to store normalized adjacency entries
adjacency_entries_1 = []
adjacency_entries_2 = []
adjacency_entries_3 = []


for conn in connections:
    point1, point2, point3 = int(conn[0]), int(conn[1]), int(conn[2])
    
    # Add the entries to the list
    adjacency_entries_1.append([point1, point2])
    adjacency_entries_2.append([point2, point3])
    adjacency_entries_3.append([point3, point1])

# Convert the lists to NumPy arrays
adjacency_array_1 = np.array(adjacency_entries_1)
adjacency_array_2 = np.array(adjacency_entries_2)
adjacency_array_3 = np.array(adjacency_entries_3)

# Combine the arrays vertically
adjacency_array = np.vstack((adjacency_array_1, adjacency_array_2, adjacency_array_3))

# Save the NumPy array to a CSV file
np.savetxt('connectivity_matrix.csv', adjacency_array, fmt='%d', delimiter=',', header='point_i,point_j', comments='')

### Write Edge_weight matrix

In [ ]:
import pandas as pd 
import numpy as np
from scipy.spatial import distance

dataset_coords = np.load('Dataset\\agard_doe_r002_dataset_reduced.npy')
dataset_coords = dataset_coords[:,:,:3]

edge_weight = []
coordinates = pd.read_csv('coordinates.csv', header=None)
point_id = np.array(coordinates[3])

# Read the file containing point connections with three columns
with open('marker_elements.csv', 'r') as conn_file:
    connections = [line.strip().split(',') for line in conn_file]

for sample in range(dataset_coords.shape[0]):
    print(sample)
    coordinate_dict = {point_id[id]: (float(dataset_coords[sample,id,0]), float(dataset_coords[sample,id,1]), float(dataset_coords[sample,id,2])) for id in range(point_id.shape[0])}

    # Create a list to store normalized adjacency entries
    adjacency_entries_1 = []
    adjacency_entries_2 = []
    adjacency_entries_3 = []

    # Find the maximum distances for normalization
    max_distance = float('-inf')

    # Populate the list with entries (point1, point2, point3, normalized_distance)
    for conn in connections:
        point1, point2, point3 = int(conn[0]), int(conn[1]), int(conn[2])
        coord1, coord2, coord3 = coordinate_dict[point1], coordinate_dict[point2], coordinate_dict[point3]
        
        # Calculate the distances between the points
        dist1 = distance.euclidean(coord1, coord2)
        dist2 = distance.euclidean(coord2, coord3)
        dist3 = distance.euclidean(coord3, coord1)
        
        # Take the inverse of the distances
        inv_dist1 = 1.0 / dist1
        inv_dist2 = 1.0 / dist2
        inv_dist3 = 1.0 / dist3
        
        # Update max distances
        max_distance = max(max_distance, inv_dist1, inv_dist2, inv_dist3)

    # Normalize and add entries to the list
    for conn in connections:
        point1, point2, point3 = int(conn[0]), int(conn[1]), int(conn[2])
        coord1, coord2, coord3 = coordinate_dict[point1], coordinate_dict[point2], coordinate_dict[point3]
        
        # Calculate the distances between the points
        dist1 = distance.euclidean(coord1, coord2)
        dist2 = distance.euclidean(coord2, coord3)
        dist3 = distance.euclidean(coord3, coord1)

        # Take the inverse of the distances
        inv_dist1 = 1.0 / dist1
        inv_dist2 = 1.0 / dist2
        inv_dist3 = 1.0 / dist3
        
        # Normalize the inverse distances between 0 and 1
        normalized_distance1 = inv_dist1 / max_distance
        normalized_distance2 = inv_dist2 / max_distance
        normalized_distance3 = inv_dist3 / max_distance
        
        # Add the entries to the list
        adjacency_entries_1.append([point1, point2, normalized_distance1])
        adjacency_entries_2.append([point2, point3, normalized_distance2])
        adjacency_entries_3.append([point3, point1, normalized_distance3])


    # Convert the lists to NumPy arrays
    adjacency_array_1 = np.array(adjacency_entries_1)
    adjacency_array_2 = np.array(adjacency_entries_2)
    adjacency_array_3 = np.array(adjacency_entries_3)

    # Combine the arrays vertically
    adjacency_array = np.vstack((adjacency_array_1, adjacency_array_2, adjacency_array_3))

    edge_weight.append(adjacency_array[:,-1])

np.save('edge_weight_array.npy',np.array(edge_weight))
print(np.array(edge_weight).shape)